In [1]:
import pandas as pd
from stage import fullPrint, getStage
from datetime import datetime, timedelta
import yahoo_fin.stock_info as yf
import numpy as np
import pickle
import swifter
import os

nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)

PEAK = 13
allList = []
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
allList = (nasdaqList + nyseList)

SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))
stockDataTickerList = []
now = datetime.now()

In [2]:
for ticker in nasdaqList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
        df = df[~df.index.duplicated()]
        dfBuy = df
        df['Dates'] = pd.to_datetime(df.index)
        df = df[df['Dates'].dt.weekday == 4]
        df = df.drop('Dates', axis = 1)
        df.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
        dfBuy['Dates'] = pd.to_datetime(dfBuy.index)
        dfBuy = dfBuy[dfBuy['Dates'].dt.weekday == 0]
        dfBuy = dfBuy.drop('Dates', axis = 1)
        dfBuy.to_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        stockDataTickerList.append(ticker)
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
except:
    print("Something went wrong")

1/4113
2/4113
3/4113
4/4113
5/4113
6/4113
7/4113
8/4113
9/4113
10/4113
11/4113
12/4113
13/4113
14/4113
15/4113
16/4113
17/4113
18/4113
19/4113
20/4113
21/4113
22/4113
23/4113
24/4113
25/4113
26/4113
27/4113
28/4113
29/4113
30/4113
31/4113
32/4113
33/4113
34/4113
35/4113
36/4113
37/4113
38/4113
39/4113
40/4113
41/4113
42/4113
43/4113
44/4113
45/4113
46/4113
47/4113
48/4113
49/4113
50/4113
51/4113
52/4113
53/4113
54/4113
55/4113
56/4113
57/4113
58/4113
59/4113
60/4113
61/4113
62/4113
63/4113
64/4113
65/4113
66/4113
67/4113
68/4113
69/4113
70/4113
71/4113
72/4113
73/4113
74/4113
75/4113
76/4113
77/4113
78/4113
79/4113
80/4113
81/4113
82/4113
83/4113
84/4113
85/4113
86/4113
87/4113
88/4113
89/4113
90/4113
91/4113
92/4113
93/4113
94/4113
95/4113
96/4113
97/4113
98/4113
99/4113
100/4113
101/4113
102/4113
103/4113
104/4113
105/4113
106/4113
107/4113
108/4113
109/4113
110/4113
111/4113
112/4113
113/4113
114/4113
115/4113
116/4113
117/4113
118/4113
119/4113
120/4113
121/4113
122/4113
123/4113
1

In [4]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,4]

def calculateSpyDfSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,8]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])


In [5]:
spDf = yf.get_data('^IXIC', start_date="1995-01-06",end_date = now, index_as_date = True)
spDf = spDf[~spDf.index.duplicated()]
spDf['close'] = spDf['close'].rolling(window=5).apply(lambda x: np.average(x))
spDf['Dates'] = pd.to_datetime(spDf.index)
spDf = spDf[spDf['Dates'].dt.weekday == 4]
spDf = spDf.drop('Dates', axis = 1)
spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf['WMA30'] = spDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
spDf['WMA30Slope'] = spDf.swifter.apply(lambda x: calculateSpyDfSlopeOfTicker(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf.to_pickle("stockData/Spy.pkl")

Pandas Apply: 100%|██████████| 1374/1374 [00:00<00:00, 26885.44it/s]


In [2]:
for ticker in stockDataTickerList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        buyDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        if stockDf['close'].isnull().values.any() or buyDf['open'].isnull().values.any():
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            os.remove('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, PEAK] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass


In [4]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-02",end_date= now, index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate = transactionTemplate[~transactionTemplate.index.duplicated()]
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [5]:
for ticker in nasdaqList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

In [6]:
now = datetime.now()
monday = now - timedelta(days = now.weekday())
now = now.strftime('%Y-%m-%d')
df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
df['Dates'] = pd.to_datetime(df.index)
df = df[df['Dates'].dt.weekday == 4]
df = df.drop('Dates', axis = 1)
df

,open,high,low,close,adjclose,volume,ticker


In [7]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-06",end_date= now, index_as_date = True).drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate['Dates'] = pd.to_datetime(transactionTemplate.index)
transactionTemplate = transactionTemplate[transactionTemplate['Dates'].dt.weekday == 4]
transactionTemplate = transactionTemplate.drop('Dates', axis = 1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [8]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            if  df.at[date,'MA30Slope'] > 0.992549883242513 and df.loc[date,'RS'] > -0.3044455098712273:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1


In [12]:
removeList = []
CLOSE = 0
TICKER = 2
for ticker in nasdaqList:
    try:
        df = pd.read_pickle("stockData/nyseNasdaq/"+ticker+".pkl")
        lastClose = df.iat[0,CLOSE]
        isFirst = True
        index = 0
        for date in df.Stage:
            if isFirst == True:
                isFirst = False
                continue
            if (df.iat[index,CLOSE] >= lastClose * 5):
                removeList.append(ticker)
                break
            lastClose = df.iat[index,CLOSE]
            index += 1
        df = getStage(ticker,[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)
        if df.empty:
            removeList.append(ticker)
    except:
        removeList.append(ticker)
print(removeList)

['ADSEW', 'AVACW', 'BZFDW', 'ENTFW', 'FIACW', 'GTACW', 'STRNW', 'TOIIW', 'TOIIW']


In [13]:
for ticker in removeList:
    nasdaqList.remove(ticker)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)

In [14]:

removeList = []
for ticker in nasdaqList:
    df = pd.read_pickle("stockData/nyseNasdaq/"+ticker+".pkl")
    lastClose = df.iat[0,CLOSE]
    isFirst = True
    index = 0
    for date in df.Stage:
        if isFirst == True:
            isFirst = False
            continue
        if (df.iat[index,CLOSE] >= lastClose * 5):
            removeList.append(ticker)
            break
        lastClose = df.iat[index,CLOSE]
        index += 1
    


In [5]:
for ticker in removeList:
    nasdaqList.remove(ticker)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)

In [ ]:
import pandas as pd
df = pd.read_pickle("stockData/nasdaq.pkl")
print(df['USEG'])

In [3]:
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
for ticker in nasdaqList:
    stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    buyDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
    if stockDf['close'].isnull().values.any() or buyDf['open'].isnull().values.any():
        print(ticker)
        removeList.append(ticker)
        os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
        os.remove('stockData/nyseNasdaq/'+ticker+'Buy.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'stockData/nyseNasdaq/CONE.pkl'

In [4]:
for ticker in removeList:
    nasdaqList.remove(ticker)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)